In [ ]:
%pip install psycopg2
%pip install nltk
%pip install transformers
from gensim.utils import tokenize
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.similarities import SparseMatrixSimilarity
import nltk
import psycopg2
from transformers import pipeline

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')

In [ ]:
# Connect to the database and fetch documents
conn = psycopg2.connect(database = "diariorepublica", 
                        user = "postgres", 
                        host= 'localhost',
                        password = "1597535",
                        port = 5432)

cur = conn.cursor()

In [ ]:
def preprocess(line):
    line = line.lower()
    tokens = tokenize(line)
    tokens = [token for token in tokens if token not in stopwords]
    return list(tokens)